# Configuration

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup(level=0)

2018-02-24 11:37:54,043 INFO    : root         : Using LISA logging configuration:
2018-02-24 11:37:54,044 INFO    : root         :   /home/leoy/Work/tools/lisa-ml-20171106/logging.conf


In [2]:
%matplotlib inline

import devlib
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import trappy

from collections import OrderedDict
from csv import DictWriter
from env import TestEnv
from matplotlib.ticker import FormatStrFormatter, MaxNLocator
from scipy.stats import linregress
from scipy import polyfit
from time import sleep
from trappy.plotter.ColorMap import ColorMap

# Support for trace events analysis
from trace import Trace
# Import support for Android devices
from android import Screen, Workload, System

2018-02-24 11:37:54,927 DEBUG   : JsonConf     : loading JSON...
2018-02-24 11:37:54,929 DEBUG   : JsonConf     : Loaded JSON configuration:
2018-02-24 11:37:54,930 DEBUG   : JsonConf     :    {u'board': {u'big_core': u'A15', u'cores': [u'A7', u'A7', u'A7', u'A7', u'A15', u'A15', u'A15', u'A15'], u'modules': [u'bl', u'cpufreq']}}
2018-02-24 11:37:54,932 DEBUG   : JsonConf     : loading JSON...
2018-02-24 11:37:54,933 DEBUG   : JsonConf     : Loaded JSON configuration:
2018-02-24 11:37:54,935 DEBUG   : JsonConf     :    {u'nrg_model': {u'big': {u'cluster': {u'nrg_max': 78}, u'cpu': {u'cap_max': 1024, u'nrg_max': 1893}}, u'little': {u'cluster': {u'nrg_max': 56}, u'cpu': {u'cap_max': 472, u'nrg_max': 215}}}, u'board': {u'big_core': u'A72', u'cores': [u'A53', u'A53', u'A53', u'A53', u'A72', u'A72'], u'modules': [u'bl', u'cpufreq', u'cpuidle']}}
2018-02-24 11:37:54,936 DEBUG   : JsonConf     : loading JSON...
2018-02-24 11:37:54,938 DEBUG   : JsonConf     : Loaded JSON configuration:
2018-0

In [3]:
# Setup a target configuration
my_conf = {
    
    # Target platform and board
    "platform"    : 'android',
    "board"       : 'hikey960',
    
    "device"      : "2450309301A1C7FB",

    "results_dir" : "hikey960_cpu_benchmark",

    # Energy Meters Configuration for BayLibre's ACME Cape
    "emeter" : {
        "instrument" : "aep",
        "conf" : {
            'resistor_values' : [0.033],
            'device_entry'    : '/dev/ttyACM0'
        },
        "channel_map" : {
            "Device0" : "BAT"
        }
    },
    
    # Tools required by the experiments
    "tools"   : ['trace-cmd', 'sysbench'],
    "modules" : ['cpufreq', 'cpuidle', 'hotplug', 'cgroups', 'thermal'],
    
    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "cpu_frequency",
            "cpu_idle",
            "sched_switch"
         ],
         "buffsize" : 200 * 1024,
    },
}

In [4]:
!adb root

adbd is already running as root


In [5]:
# Initialize a test environment using:
# the provided target configuration (my_conf)
te = TestEnv(target_conf=my_conf, force_new=True)
target = te.target

2018-02-24 11:37:55,525 INFO    : TestEnv      : Using base path: /home/leoy/Work/tools/lisa-ml-20171106
2018-02-24 11:37:55,530 INFO    : TestEnv      : Loading custom (inline) target configuration
2018-02-24 11:37:55,533 WARNING : TestEnv      : Wipe previous contents of the results folder:
2018-02-24 11:37:55,535 WARNING : TestEnv      :    /home/leoy/Work/tools/lisa-ml-20171106/results/hikey960_cpu_benchmark
2018-02-24 11:37:55,537 INFO    : TestEnv      : External tools using:
2018-02-24 11:37:55,538 INFO    : TestEnv      :    ANDROID_HOME: /home/leoy/Android/Sdk/
2018-02-24 11:37:55,539 INFO    : TestEnv      :    CATAPULT_HOME: /home/leoy/Work/tools/lisa-ml-20171106/tools/catapult
2018-02-24 11:37:55,541 INFO    : TestEnv      : Devlib modules to load: ['cgroups', 'cpufreq', 'bl', 'hotplug', 'thermal', 'cpuidle']
2018-02-24 11:37:55,542 INFO    : TestEnv      : Connecting Android target [2450309301A1C7FB]
2018-02-24 11:37:55,543 INFO    : TestEnv      : Connection settings:
201

2018-02-24 11:38:02,653 DEBUG   : CGroup.debug : Controller debug mounted under: /data/local/tmp/devlib-target/cgroups/devlib_cgh6 (noprefix=False)
2018-02-24 11:38:02,657 DEBUG   : cgroups.debug : Creating cgroup /data/local/tmp/devlib-target/cgroups/devlib_cgh6
2018-02-24 11:38:02,777 INFO    : CGroups      :   debug        : /data/local/tmp/devlib-target/cgroups/devlib_cgh6
2018-02-24 11:38:04,038 INFO    : TestEnv      : Attempting to read energy model from target
2018-02-24 11:38:05,843 INFO    : TestEnv      : Topology:
2018-02-24 11:38:05,845 INFO    : TestEnv      :    [[0, 1, 2, 3], [4, 5, 6, 7]]
2018-02-24 11:38:06,031 INFO    : TestEnv      : Loading default EM:
2018-02-24 11:38:06,033 INFO    : TestEnv      :    /home/leoy/Work/tools/lisa-ml-20171106/libs/utils/platforms/hikey960.json
2018-02-24 11:38:06,037 DEBUG   : JsonConf     : loading JSON...
2018-02-24 11:38:06,045 DEBUG   : JsonConf     : Loaded JSON configuration:
2018-02-24 11:38:06,046 DEBUG   : JsonConf     :   

## Energy Model Parameters (CPUs, OPPs and Idle States)
Cluster data will be extracted automatically from the target device.

### Platform cluster layout discovery

In [6]:
clusters = {}

for cpu, cluster in enumerate(target.core_clusters):
    if cluster not in clusters:
        clusters[cluster] = {}
        clusters[cluster]["cpus"] = []
        
    clusters[cluster]["cpus"].append(cpu)
    
for cluster in clusters.itervalues():
    first_cpu = cluster["cpus"][0]
    
    cluster["freqs"] = target.cpufreq.list_frequencies(first_cpu)
    cluster["idles"] = range(len(target.cpuidle.get_states(first_cpu)))

In [7]:
for cluster, values in clusters.iteritems():
    print "===== cluster {} =====".format(cluster)
    print "CPUs {}".format(values["cpus"])
    print "freqs {}".format(values["freqs"])
    print "idle states {}".format(values["idles"])

===== cluster 0 =====
CPUs [0, 1, 2, 3]
freqs [533000, 999000, 1402000, 1709000, 1844000]
idle states [0, 1, 2]
===== cluster 1 =====
CPUs [4, 5, 6, 7]
freqs [903000, 1421000, 1805000, 2112000, 2362000]
idle states [0, 1, 2]


## Device setup

### Thermal management
Thermal management will be disabled (at least the software part, there quite likely is some sort of in-hardware/firmware protection) to not impact the energy readings. If your device overheats easily, you could try using a fan to cool it.

In [8]:
target.thermal.disable_all_zones()

### Temperature sensor
A temperature sensor is needed to ensure each test iteration is run in roughly the same conditions (i.e. the same temperature). In this example we'll use the sensor used by a thermal zone, but any other method could be used as long as a **get_temperature()** method can be called.

In [9]:
temperature_sensor = target.thermal.zones[0]

## Benchmark example
In this section, we setup classes that will give us an API to trigger and then analyse any benchmarking application

In [37]:
class EM_workload(object):
    """
    Energy Model workload class
    
    :param target: Devlib target to run the workload on
    :type target: devlib.target
    
    :param binary: Path to the workload binary on the target
    :type binary: str
    
    :param duration: Execution duration of the workload
    :type duration: int
    
    :param args: Additional arguments to be given to the binary call
    :type args: str or list(str)
    """
    
    def __init__(self, target, binary, duration, args=None):
        self.target = target
        self.binary = binary
        self.duration = duration
        self.args = args
        
    def run(self, cpus):
        """
        Launch len('cpus') executions of 'binary', one per cpu
        
        :param cpus: CPU list of CPUs that will run this workload
        :type cpus: list(int)
        
        :returns: The cumulative score of each workload as a dict(cpu : score)
        """
        self.procs = {}
        for cpu in cpus:
            self.procs[cpu] = self.target.background_invoke(binary=self.binary, args=self.args, on_cpus=[cpu])
        
        #sleep(2)
        
        stderr = {}
        stdout = {}
        for cpu in cpus:
            stderr[cpu], stdout[cpu] = self.procs[cpu].communicate()
            print stderr[cpu]
            print stdout[cpu]
            
        score = {}
        for cpu in cpus:
            print stderr[cpu]
            score[cpu] = self._parse_score(stderr[cpu], stdout[cpu])
        
        return score
    
    def _parse_score(self, stderr, stdout):
        raise NotImplemented("_parse_score must be implemented")

In [38]:
class EM_dhrystone(EM_workload):
    
    def __init__(self, target, binary, duration, args=None):
        EM_workload.__init__(self, target, binary, duration)
        self.args = '-l {}'.format(duration)

    def _parse_score(self, stderr, stdout):
        match = re.search('This machine benchmarks at \s*(?P<score>[0-9]+) dhrystones/second', stderr)
        return float(match.group('score'))

In [39]:
class EM_dhrystone2_1(EM_workload):
    
    def __init__(self, target, binary, duration, args=None):
        EM_workload.__init__(self, target, binary, duration)
        self.args = '-l {}'.format(duration)

    def _parse_score(self, stderr, stdout):
        match = re.search('Dhrystones per Second:\s*(?P<score>[0-9]+)', stderr)
        return float(match.group('score'))

In [40]:
class EM_sysbench(EM_workload):
    
    def __init__(self, target, binary, duration, args=None):
        EM_workload.__init__(self, target, binary, duration)
        self.args = '--test=cpu --max-time={} run'.format(duration)

    def _parse_score(self, stderr, stdout):
        match = re.search('total number of events:\s*(?P<events>[0-9]+)', stderr)
        return float(match.group('events'))

## Performance analysis
In this section, we will look at the performance characteristics of the platform.

In [41]:
class perf_recorder(object):
    """
    Data collector for C-states analysis
    
    :param clusters: Dictionnary describing the clusters to analyze
        Format: {
            cluster_name : {
                "cpus"  : [0, 1],
                "freqs" : [100, 200]
            }
        }
    :type clusters: dict
    
    :param benchmark: instance of an EM_workload subclass
    :type benchmark: EM_workload
    
    """
    
    def __init__(self, clusters, benchmark):
        self.clusters = clusters
        self.benchmark = benchmark
        
    @property
    def online_cpus(self):
        return target.list_online_cpus()

    def _read_perf(self):
        energy = 0.0
        perf = { cpu : 0.0 for cpu in self.online_cpus }

        for i in xrange(self.loop_cnt):

            for cpu, value in self.benchmark.run(self.online_cpus).iteritems():
                perf[cpu] += value
            
        for cpu in perf.iterkeys():
            perf[cpu] = perf[cpu] / self.loop_cnt
            
        return perf
    
    def _set_and_check_freq(self, cpu, freq):
        target.cpufreq.set_frequency(cpu, freq)
        rfreq = target.cpufreq.get_frequency(cpu)

        if rfreq != freq:
            raise RuntimeError("Requested freq {}, got {}".format(freq, rfreq))
    
    def _analysis_setup(self):
        # Make sure all CPUs are online
        target.hotplug.online_all()
        # Set cpufreq governor to userpace to allow manual frequency scaling
        target.cpufreq.set_all_governors('userspace')
        
        self.data = []
        
        # Prepare backup file writer
        self._fh = open(self.bkp_file, 'w')
        self._writer = DictWriter(
            self._fh,
            fieldnames=['cluster', 'cpus', 'freq', 'Dhrystones per Second', 'DMIPS/MHz']
        )
        
        # Read current temperature
        self.test_temperature = 0
        for i in range(10):
            self.test_temperature += temperature_sensor.get_temperature()
        self.test_temperature /= 10
        
        logging.info("Test ambient temperature determined as: {}".format(self.test_temperature))
            
        
    def _analysis_teardown(self):
        target.hotplug.online_all()
        
        # Cleanup backup fh
        self._fh.close()
        

    def _setup_analysis_cluster(self, cluster_idx):

        self.cur_cluster = cluster_idx
        online_cluster = self.clusters[cluster_idx]
        online_cpu = online_cluster["cpus"][0]
        
        # Make sure one core of the cluster is online
        target.hotplug.online(online_cpu)
        
        # Offline all other cores
        for cluster in self.clusters.itervalues():
            for cpu in cluster["cpus"]:
                if cpu != online_cpu:
                    target.hotplug.offline(cpu)
                    
        # See if we can use another cluster for energy comparison
        if len(self.clusters) < 2:
            self.noise_cluster = -1
            self.noise_cpu = -1
        else:
            self.noise_cluster = (cluster_idx + 1) % len(self.clusters)
            self.noise_cpu = self.clusters[self.noise_cluster]["cpus"][0]                    
      
    def _setup_analysis_cpu(self, cpu):
        target.hotplug.online(cpu)
        
    def _analyse(self, freq):
        self._set_and_check_freq(self.clusters[self.cur_cluster]["cpus"][0], freq)
        
        perf = self._read_perf()
        # Remove performance result from noise-measurement CPU
        perf.pop(self.noise_cpu, None)
        
        res = {
            'cluster': self.cur_cluster,
            'cpus': len(perf),
            'freq': freq,
            'Dhrystones per Second': sum(perf.values()),
            'DMIPS/MHz': sum(perf.values()) / 1757 / (freq/1000)
        }
        
        # Save data to backup file
        self._writer.writerow(res)
        
        # Save data locally
        self.data.append(res)
    
    def run_analysis(self, loop_cnt, bkp_file='pstates.csv'):
        """
        This will record energy values for the P-states of the target.
        Energy will be recorded with an increasing amount of active CPUs (from 1 to all of the cluster's CPUs),
        for all available frequencies. This will be done on each cluster.
        
        :param loop_cnt: Number of iterations for each benchmark execution
        :type loop_cnt: int
        
        :param bkp_file: File in which the recorded data will be appended
            Useful to resume the analysis after closing the Notebook
        :type bkp_file: str
        """
        self.loop_cnt = loop_cnt
        self.bkp_file = bkp_file
        
        try:
            self._analysis_setup()
            # A) For each cluster
            for cluster_idx in self.clusters.iterkeys():
                logging.info("Cluster={}".format(cluster_idx))
                self._setup_analysis_cluster(cluster_idx)
                # B) For each core of that cluster
                self._setup_analysis_cpu(clusters[cluster_idx]["cpus"][0])
                logging.info("Online cpus={}".format(self.online_cpus))
                    
                # C) For each supported OPP
                for freq in clusters[cluster_idx]["freqs"]:
                    logging.info("Freq={}".format(freq))
                    self._analyse(freq)
        finally:
            self._analysis_teardown()
        
        df = pd.DataFrame(self.data)

        return df.set_index(['cluster', 'freq', 'cpus']).sort_index(level='cluster')

In [42]:
#sysbench = EM_sysbench(target, target.which("sysbench"), 5)
#pp_recorder = power_perf_recorder(clusters, sysbench)

In [43]:
#dhry = EM_dhrystone(target, "/data/dhrystone", 20)
#pp_recorder = perf_recorder(clusters, dhry)

# Dhrystone2.1 Performance result

In [44]:
dhry2_1 = EM_dhrystone2_1(target, "/data/gcc_dry2", 20)
pp_recorder = perf_recorder(clusters, dhry2_1)

In [45]:
power_perf = pp_recorder.run_analysis(loop_cnt=2)

2018-02-24 11:44:54,899 INFO    : root         : Test ambient temperature determined as: 46675
2018-02-24 11:44:54,901 INFO    : root         : Cluster=0
2018-02-24 11:44:58,477 INFO    : root         : Online cpus=[0]
2018-02-24 11:44:58,479 INFO    : root         : Freq=533000



Dhrystone Benchmark, Version 2.1 (Language: C)

Program compiled without 'register' attribute

Please give the number of runs through the benchmark: 
Execution starts, 50000000 runs through Dhrystone
Execution ends

Final values of the variables used in the benchmark:

Int_Glob:            5
        should be:   5
Bool_Glob:           1
        should be:   1
Ch_1_Glob:           A
        should be:   A
Ch_2_Glob:           B
        should be:   B
Arr_1_Glob[8]:       7
        should be:   7
Arr_2_Glob[8][7]:    50000010
        should be:   Number_Of_Runs + 10
Ptr_Glob->
  Ptr_Comp:          362248656
        should be:   (implementation-dependent)
  Discr:             0
        should be:   0
  Enum_Comp:         2
        should be:   2
  Int_Comp:          17
        should be:   17
  Str_Comp:          DHRYSTONE PROGRAM, SOME STRING
        should be:   DHRYSTONE PROGRAM, SOME STRING
Next_Ptr_Glob->
  Ptr_Comp:          362248656
        should be:   (implementation-dependent)

2018-02-24 11:45:51,927 INFO    : root         : Freq=999000



Dhrystone Benchmark, Version 2.1 (Language: C)

Program compiled without 'register' attribute

Please give the number of runs through the benchmark: 
Execution starts, 50000000 runs through Dhrystone
Execution ends

Final values of the variables used in the benchmark:

Int_Glob:            5
        should be:   5
Bool_Glob:           1
        should be:   1
Ch_1_Glob:           A
        should be:   A
Ch_2_Glob:           B
        should be:   B
Arr_1_Glob[8]:       7
        should be:   7
Arr_2_Glob[8][7]:    50000010
        should be:   Number_Of_Runs + 10
Ptr_Glob->
  Ptr_Comp:          739129808
        should be:   (implementation-dependent)
  Discr:             0
        should be:   0
  Enum_Comp:         2
        should be:   2
  Int_Comp:          17
        should be:   17
  Str_Comp:          DHRYSTONE PROGRAM, SOME STRING
        should be:   DHRYSTONE PROGRAM, SOME STRING
Next_Ptr_Glob->
  Ptr_Comp:          739129808
        should be:   (implementation-dependent)

2018-02-24 11:46:20,504 INFO    : root         : Freq=1402000


...


Dhrystone Benchmark, Version 2.1 (Language: C)

Program compiled without 'register' attribute

Please give the number of runs through the benchmark: 
Execution starts, 50000000 runs through Dhrystone
Execution ends

Final values of the variables used in the benchmark:

Int_Glob:            5
        should be:   5
Bool_Glob:           1
        should be:   1
Ch_1_Glob:           A
        should be:   A
Ch_2_Glob:           B
        should be:   B
Arr_1_Glob[8]:       7
        should be:   7
Arr_2_Glob[8][7]:    50000010
        should be:   Number_Of_Runs + 10
Ptr_Glob->
  Ptr_Comp:          506526160
        should be:   (implementation-dependent)
  Discr:             0
        should be:   0
  Enum_Comp:         2
        should be:   2
  Int_Comp:          17
        should be:   17
  Str_Comp:          DHRYSTONE PROGRAM, SOME STRING
        should be:   DHRYSTONE PROGRAM, SOME STRING
Next_Ptr_Glob->
  Ptr_Comp:          506526160
        should be:   (implementation-dependent)

2018-02-24 11:47:52,299 INFO    : root         : Freq=2112000



Dhrystone Benchmark, Version 2.1 (Language: C)

Program compiled without 'register' attribute

Please give the number of runs through the benchmark: 
Execution starts, 50000000 runs through Dhrystone
Execution ends

Final values of the variables used in the benchmark:

Int_Glob:            5
        should be:   5
Bool_Glob:           1
        should be:   1
Ch_1_Glob:           A
        should be:   A
Ch_2_Glob:           B
        should be:   B
Arr_1_Glob[8]:       7
        should be:   7
Arr_2_Glob[8][7]:    50000010
        should be:   Number_Of_Runs + 10
Ptr_Glob->
  Ptr_Comp:          79301072
        should be:   (implementation-dependent)
  Discr:             0
        should be:   0
  Enum_Comp:         2
        should be:   2
  Int_Comp:          17
        should be:   17
  Str_Comp:          DHRYSTONE PROGRAM, SOME STRING
        should be:   DHRYSTONE PROGRAM, SOME STRING
Next_Ptr_Glob->
  Ptr_Comp:          79301072
        should be:   (implementation-dependent), 

2018-02-24 11:48:00,031 INFO    : root         : Freq=2362000



Dhrystone Benchmark, Version 2.1 (Language: C)

Program compiled without 'register' attribute

Please give the number of runs through the benchmark: 
Execution starts, 50000000 runs through Dhrystone
Execution ends

Final values of the variables used in the benchmark:

Int_Glob:            5
        should be:   5
Bool_Glob:           1
        should be:   1
Ch_1_Glob:           A
        should be:   A
Ch_2_Glob:           B
        should be:   B
Arr_1_Glob[8]:       7
        should be:   7
Arr_2_Glob[8][7]:    50000010
        should be:   Number_Of_Runs + 10
Ptr_Glob->
  Ptr_Comp:          690764240
        should be:   (implementation-dependent)
  Discr:             0
        should be:   0
  Enum_Comp:         2
        should be:   2
  Int_Comp:          17
        should be:   17
  Str_Comp:          DHRYSTONE PROGRAM, SOME STRING
        should be:   DHRYSTONE PROGRAM, SOME STRING
Next_Ptr_Glob->
  Ptr_Comp:          690764240
        should be:   (implementation-dependent)

In [46]:
power_perf

DMIPS/MHz  Dhrystones per Second
cluster freq    cpus                                  
0       533000  1      2.015485              1887463.5
        999000  1      2.034720              3571428.0
        1402000 1      2.029786              5000000.0
        1709000 1      2.081451              6250000.0
        1844000 1      2.066858              6696428.5
1       903000  1      3.501611              5555555.0
        1421000 1      3.671517              9166666.5
        1805000 1      3.547346             11250000.0
        2112000 1      3.368560             12500000.0
        2362000 1      3.514027             14583333.5